In [2]:
import pandas as pd
import numpy as np

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

In [6]:
import matplotlib.pyplot as plt

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout,Activation

from keras.layers.convolutional import Conv1D
from keras import backend as K
from keras.layers.core import Lambda
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding

import keras
from keras.utils import np_utils

In [16]:
from sklearn.model_selection import GridSearchCV

In [70]:
from sklearn.feature_selection import VarianceThreshold

In [67]:
dataset = pd.read_csv('Usman_test_train_IoT/Combined_Data_IoT.csv')

In [71]:
dataset.columns

Index(['ts', 'date', 'time', 'fridge_temperature', 'temp_condition', 'label',
       'type', 'door_state', 'sphone_signal', 'latitude', 'longitude',
       'FC1_Read_Input_Register', 'FC2_Read_Discrete_Value',
       'FC3_Read_Holding_Register', 'FC4_Read_Coil', 'motion_status',
       'light_status', 'current_temperature', 'thermostat_status',
       'temperature', 'pressure', 'humidity'],
      dtype='object')

In [72]:
Data = dataset.drop(['ts', 'date', 'time'], axis =1)

In [76]:
X = Data.drop(['label', 'type'], axis = 1)
X.shape

(401119, 17)

In [77]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8))) #don't change these values
x_new = sel.fit(X)

In [79]:
cols1 = x_new.get_support(indices=True)
X_F = X.iloc[:,cols1]
X_F

,fridge_temperature,latitude,longitude,FC1_Read_Input_Register,FC2_Read_Discrete_Value,FC3_Read_Holding_Register,FC4_Read_Coil,current_temperature,temperature,pressure,humidity
0,9.00,4.514077,14.421946,32450,32708,32035,32728,28.442693,35.773605,1.035000,46.343618
1,9.25,4.514077,14.421946,32450,32708,32035,32728,28.442693,35.773605,1.035000,46.343618
2,12.65,4.514077,14.421946,32450,32708,32035,32728,28.442693,35.773605,1.035000,46.343618
3,4.65,4.514077,14.421946,32450,32708,32035,32728,28.442693,35.773605,1.035000,46.343618
4,12.65,4.514077,14.421946,32450,32708,32035,32728,28.442693,35.773605,1.035000,46.343618
...,...,...,...,...,...,...,...,...,...,...,...
401114,6.70,4.514077,14.421946,32450,32708,32035,32728,28.442693,32.799434,2.204924,37.024913
401115,6.70,4.514077,14.421946,32450,32708,32035,32728,28.442693,29.453781,-2.030547,90.297894
401116,6.70,4.514077,14.421946,32450,32708,32035,32728,28.442693,47.185992,0.872942,37.687701
401117,6.70,4.514077,14.421946,32450,32708,32035,32728,28.442693,43.097037,3.168207,93.647950


In [88]:
scaler = MinMaxScaler()
#XS = scaler.fit_transform(X_F)
XS = scaler.fit_transform(X)
print(XS)

[[0.61538462 1.         0.         ... 0.51730735 0.53355618 0.46251056]
 [0.63461538 1.         0.         ... 0.51730735 0.53355618 0.46251056]
 [0.89615385 1.         0.         ... 0.51730735 0.53355618 0.46251056]
 ...
 [0.43846154 1.         0.         ... 0.90452247 0.52695586 0.3756012 ]
 [0.43846154 1.         0.         ... 0.76578681 0.62043775 0.93746771]
 [0.43846154 1.         0.         ... 0.40588822 0.58120503 0.3689465 ]]


In [89]:
XS.shape

(401119, 17)

In [90]:
y = Data['label']
y = y.values
y.shape

(401119,)

In [91]:
(trainX, testX, trainY, testY) = train_test_split(XS, y, test_size = 0.20, random_state = 5)

In [92]:
#Stratified K Fold split
kfold = StratifiedKFold(n_splits=4)
print(kfold)

StratifiedKFold(n_splits=4, random_state=None, shuffle=False)


In [93]:
print(X.shape)
print(y.shape)
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(401119, 17)
(401119,)
(320895, 17)
(320895,)
(80224, 17)
(80224,)


# Model Development (Binary)

### LR

In [94]:
#Test Train split
lr = LogisticRegression() #for binary
lr.fit(trainX, trainY)
y_predict = lr.predict(testX)
print(accuracy_score(testY, y_predict))
print(classification_report(testY, y_predict))

0.6293627842042282
              precision    recall  f1-score   support

           0       0.63      0.98      0.76     48919
           1       0.70      0.09      0.16     31305

    accuracy                           0.63     80224
   macro avg       0.66      0.53      0.46     80224
weighted avg       0.65      0.63      0.53     80224



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [32]:
#Kfold Split
lr = LogisticRegression()
cv_results = cross_val_score(lr, XS, y, cv=kfold, scoring='accuracy')
cv_results_mean = cv_results.mean()
cv_results_std = cv_results.std()
cv_results_name = 'LR'
print('%s: %f (%f)' % (cv_results_name, cv_results_mean, cv_results_std))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR: 0.607658 (0.005429)


In [21]:
#Grid Search CV
model = LogisticRegression() 

parameters = [{'solver': ['newton-cg', 'lbfgs', 'liblinear'], 'C': [100, 10, 1.0, 0.1, 0.01]}]

grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 4, n_jobs = -1)

grid_search = grid_search.fit(trainX, trainY)

print('Best Results with Grid Search:')
print(grid_search.best_score_)
print(grid_search.best_params_)

y_pred = grid_search.predict(testX)

print('\nAccuracy Score  on test data: ' + str(accuracy_score(testY, y_pred)))
print(confusion_matrix(testY,y_pred))
print(classification_report(testY,y_pred))

Best Results with Grid Search:
0.6302373062600342
{'C': 1.0, 'solver': 'liblinear'}

Accuracy Score  on test data: 0.6294001794974072
[[47697  1222]
 [28509  2796]]
              precision    recall  f1-score   support

           0       0.63      0.98      0.76     48919
           1       0.70      0.09      0.16     31305

    accuracy                           0.63     80224
   macro avg       0.66      0.53      0.46     80224
weighted avg       0.65      0.63      0.53     80224



### LDA

In [58]:
#Test Train split
lda = LinearDiscriminantAnalysis()
lda.fit(trainX, trainY)
y_predict = lda.predict(testX)
print(accuracy_score(testY, y_predict))
print(classification_report(testY, y_predict))

0.6123479258077383
              precision    recall  f1-score   support

           0       0.61      1.00      0.76     48919
           1       0.98      0.01      0.01     31305

    accuracy                           0.61     80224
   macro avg       0.80      0.50      0.39     80224
weighted avg       0.76      0.61      0.47     80224



In [33]:
#Kfold Split
lda = LinearDiscriminantAnalysis()
cv_results = cross_val_score(lda, XS, y, cv=kfold, scoring='accuracy')
cv_results_mean = cv_results.mean()
cv_results_std = cv_results.std()
cv_results_name = 'LDA'
print('%s: %f (%f)' % (cv_results_name, cv_results_mean, cv_results_std))

LDA: 0.618093 (0.093511)


In [34]:
#Grid search CV
model = LinearDiscriminantAnalysis() 

parameters = [{'solver': ['svd', 'lsqr', 'eigen']}]

grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 4, n_jobs = -1)

grid_search = grid_search.fit(trainX, trainY)

print('Best Results with Grid Search:')
print(grid_search.best_score_)
print(grid_search.best_params_)

y_pred = grid_search.predict(testX)

print('\nAccuracy Score  on test data: ' + str(accuracy_score(testY, y_pred)))
print(confusion_matrix(testY,y_pred))
print(classification_report(testY,y_pred))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.62600228 0.62600539        nan]
  warnings.warn(


Best Results with Grid Search:
0.6260053929184142
{'solver': 'lsqr'}

Accuracy Score  on test data: 0.6251121858795373
[[47907  1012]
 [29063  2242]]
              precision    recall  f1-score   support

           0       0.62      0.98      0.76     48919
           1       0.69      0.07      0.13     31305

    accuracy                           0.63     80224
   macro avg       0.66      0.53      0.45     80224
weighted avg       0.65      0.63      0.51     80224



### KNN

In [28]:
#Test Train split
#default parameters used in base paper (n_neighbors, default=5) (p, default=2 for Euclidean Distance)
knn = KNeighborsClassifier()
knn.fit(trainX, trainY)
y_predict = knn.predict(testX)
print(accuracy_score(testY, y_predict))
print(classification_report(testY, y_predict))

0.6672691463901077
              precision    recall  f1-score   support

           0       0.76      0.66      0.71     48919
           1       0.56      0.68      0.62     31305

    accuracy                           0.67     80224
   macro avg       0.66      0.67      0.66     80224
weighted avg       0.68      0.67      0.67     80224



In [ ]:
#Kfold Split
knn = KNeighborsClassifier()
cv_results = cross_val_score(knn, XS, y, cv=kfold, scoring='accuracy')
cv_results_mean = cv_results.mean()
cv_results_std = cv_results.std()
cv_results_name = 'KNN'
print('%s: %f (%f)' % (cv_results_name, cv_results_mean, cv_results_std))

In [ ]:
#Grid search CV
model = KNeighborsClassifier()

parameters = [{'n_neighbors': [range(1, 21, 2)], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']}]

grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 4, n_jobs = -1)

grid_search = grid_search.fit(trainX, trainY)

print('Best Results with Grid Search:')
print(grid_search.best_score_)
print(grid_search.best_params_)

y_pred = grid_search.predict(testX)

print('\nAccuracy Score  on test data: ' + str(accuracy_score(testY, y_pred)))
print(confusion_matrix(testY,y_pred))
print(classification_report(testY,y_pred))

### RF

In [86]:
#Test Train split
rf = RandomForestClassifier(n_estimators=10,criterion='gini')
rf.fit(trainX, trainY)
y_predict1 = rf.predict(testX)
print(accuracy_score(testY, y_predict1))
print(classification_report(testY, y_predict1))

0.7471205624252094
              precision    recall  f1-score   support

           0       0.72      0.95      0.82     48919
           1       0.86      0.42      0.57     31305

    accuracy                           0.75     80224
   macro avg       0.79      0.69      0.69     80224
weighted avg       0.77      0.75      0.72     80224



In [36]:
#Kfold Split
rf = RandomForestClassifier(n_estimators=10,criterion='gini')
cv_results = cross_val_score(rf, XS, y, cv=kfold, scoring='accuracy')
cv_results_mean = cv_results.mean()
cv_results_std = cv_results.std()
cv_results_name = 'RF'
print('%s: %f (%f)' % (cv_results_name, cv_results_mean, cv_results_std))

RF: 0.587374 (0.008999)


In [37]:
#Grid Search CV
model = RandomForestClassifier()

parameters = [{'n_estimators': [10, 20, 50, 100], 'min_samples_split':[2, 5, 10],
              'min_samples_leaf':[1, 2, 4], 'criterion': ['gini','entropy']}]


grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 4, n_jobs = -1)

grid_search = grid_search.fit(trainX, trainY)

print('Best Results with Grid Search:')
print(grid_search.best_score_)
print(grid_search.best_params_)

y_pred = grid_search.predict(testX)

print('\nAccuracy Score  on test data: ' + str(accuracy_score(testY, y_pred)))
print(confusion_matrix(testY,y_pred))
print(classification_report(testY,y_pred))

Best Results with Grid Search:
0.7439598602911014
{'criterion': 'entropy', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}

Accuracy Score  on test data: 0.748741025129637
[[46912  2007]
 [18150 13155]]
              precision    recall  f1-score   support

           0       0.72      0.96      0.82     48919
           1       0.87      0.42      0.57     31305

    accuracy                           0.75     80224
   macro avg       0.79      0.69      0.69     80224
weighted avg       0.78      0.75      0.72     80224



### DT (CART)

In [30]:
#Test Train Split
dt = DecisionTreeClassifier(criterion='gini')
dt.fit(trainX, trainY)
y_predict1 = dt.predict(testX)
print(accuracy_score(testY, y_predict1))
print(classification_report(testY, y_predict1))

0.7431068009573195
              precision    recall  f1-score   support

           0       0.72      0.94      0.82     48919
           1       0.83      0.43      0.57     31305

    accuracy                           0.74     80224
   macro avg       0.78      0.69      0.69     80224
weighted avg       0.76      0.74      0.72     80224



In [ ]:
#Kfold Split
dt = DecisionTreeClassifier(criterion='gini')
cv_results = cross_val_score(dt, XS, y, cv=kfold, scoring='accuracy')
cv_results_mean = cv_results.mean()
cv_results_std = cv_results.std()
cv_results_name = 'DT-(CART)'
print('%s: %f (%f)' % (cv_results_name, cv_results_mean, cv_results_std))

In [ ]:
model = DecisionTreeClassifier() 

parameters = [{'min_samples_split': [2,5,10], 'min_samples_leaf': [1,2,4], 'splitter': ['best','random'],
              'criterion': ['gini', 'entropy']}]

grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 4, n_jobs = -1)

grid_search = grid_search.fit(trainX, trainY)

print('Best Results with Grid Search:')
print(grid_search.best_score_)
print(grid_search.best_params_)

y_pred = grid_search.predict(testX)

print('\nAccuracy Score  on test data: ' + str(accuracy_score(testY, y_pred)))
print(confusion_matrix(testY,y_pred))
print(classification_report(testY,y_pred))

### NB

In [31]:
#Test Train Split
nb = GaussianNB()
nb.fit(trainX, trainY)
y_predict1 = nb.predict(testX)
print(accuracy_score(testY, y_predict1))
print(classification_report(testY, y_predict1))

0.6518498205025928
              precision    recall  f1-score   support

           0       0.65      0.93      0.76     48919
           1       0.66      0.22      0.33     31305

    accuracy                           0.65     80224
   macro avg       0.66      0.57      0.55     80224
weighted avg       0.65      0.65      0.60     80224



In [ ]:
#Kfold Split
nb = GaussianNB()
cv_results = cross_val_score(nb, XS, y, cv=kfold, scoring='accuracy')
cv_results_mean = cv_results.mean()
cv_results_std = cv_results.std()
cv_results_name = 'NB'
print('%s: %f (%f)' % (cv_results_name, cv_results_mean, cv_results_std))

### SVM

In [72]:
#Test Train Split
svclassifier = SVC(kernel='rbf', gamma='auto')
svclassifier.fit(trainX, trainY)
y_predict = svclassifier.predict(testX)
print(accuracy_score(testY, y_predict))
print(classification_report(testY, y_predict))

0.6289265057838054
              precision    recall  f1-score   support

           0       0.63      0.98      0.76     48919
           1       0.69      0.09      0.16     31305

    accuracy                           0.63     80224
   macro avg       0.66      0.53      0.46     80224
weighted avg       0.65      0.63      0.53     80224



In [ ]:
#Kfold Split
svclassifier = SVC(kernel='rbf', gamma='auto')
cv_results = cross_val_score(svclassifier, XS, y, cv=kfold, scoring='accuracy')
cv_results_mean = cv_results.mean()
cv_results_std = cv_results.std()
cv_results_name = 'LDA'
print('%s: %f (%f)' % (cv_results_name, cv_results_mean, cv_results_std))

In [ ]:
#Grid Search CV
model = SVC() 

parameters = [{'C': [0.1,1,10], 'gamma':[0.01,0.5,1]}]

grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 4, n_jobs = -1)

grid_search = grid_search.fit(trainX, trainY)

print('Best Results with Grid Search:')
print(grid_search.best_score_)
print(grid_search.best_params_)

y_pred = grid_search.predict(testX)

print('\nAccuracy Score  on test data: ' + str(accuracy_score(testY, y_pred)))
print(confusion_matrix(testY,y_pred))
print(classification_report(testY,y_pred))

### LSTM

In [95]:
XS.shape

(401119, 17)

In [96]:
X1 = XS.reshape((-1, 1, 17))

In [97]:
X1.shape

(401119, 1, 17)

In [98]:
(trainX, testX, trainY, testY) = train_test_split(X1, y, test_size = 0.20, random_state = 5)

In [55]:
# 1 LSTM Layer (input), 3 Dense Hidden Layers
model = Sequential()
model.add(LSTM(17, input_shape=(1, 17), activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(trainX, trainY, epochs=35, batch_size=64, verbose=2, validation_data=(testX, testY))

Epoch 1/35
5014/5014 - 17s - loss: 0.6496 - accuracy: 0.6409 - val_loss: 0.6297 - val_accuracy: 0.6593
Epoch 2/35
5014/5014 - 15s - loss: 0.6313 - accuracy: 0.6537 - val_loss: 0.6259 - val_accuracy: 0.6615
Epoch 3/35
5014/5014 - 13s - loss: 0.6206 - accuracy: 0.6541 - val_loss: 0.6020 - val_accuracy: 0.6670
Epoch 4/35
5014/5014 - 13s - loss: 0.6126 - accuracy: 0.6585 - val_loss: 0.5963 - val_accuracy: 0.6700
Epoch 5/35
5014/5014 - 14s - loss: 0.6070 - accuracy: 0.6631 - val_loss: 0.5936 - val_accuracy: 0.6736
Epoch 6/35
5014/5014 - 14s - loss: 0.6043 - accuracy: 0.6645 - val_loss: 0.5900 - val_accuracy: 0.6769
Epoch 7/35
5014/5014 - 14s - loss: 0.6020 - accuracy: 0.6659 - val_loss: 0.5883 - val_accuracy: 0.6787
Epoch 8/35
5014/5014 - 15s - loss: 0.6008 - accuracy: 0.6673 - val_loss: 0.6005 - val_accuracy: 0.6732
Epoch 9/35
5014/5014 - 16s - loss: 0.6001 - accuracy: 0.6670 - val_loss: 0.5902 - val_accuracy: 0.6759
Epoch 10/35
5014/5014 - 15s - loss: 0.5990 - accuracy: 0.6675 - val_loss:

In [56]:
loss, accuracy = model.evaluate(testX, testY)
print("Loss:" + str(loss))
print("Accuracy:" + str(accuracy))

2507/2507 [==============================] - 4s 2ms/step - loss: 0.5779 - accuracy: 0.6798
Loss:0.5779118537902832
Accuracy:0.679796576499939


In [57]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 17)                2380      
_________________________________________________________________
dropout_4 (Dropout)          (None, 17)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               2304      
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               12900     
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)               

### CNN (ConvID)

In [ ]:
XS.shape

In [ ]:
X1 = XS.reshape((-1, 1, 17))

In [ ]:
X1.shape

In [26]:
(trainX, testX, trainY, testY) = train_test_split(X1, y, test_size = 0.20, random_state = 5)

In [42]:
nb_filter = 250
filter_length = 3

model = Sequential()
model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, padding='same', activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(trainX, trainY, epochs=35, batch_size=64, verbose=2, validation_data=(testX, testY))

Epoch 1/35
5014/5014 - 46s - loss: 0.6476 - accuracy: 0.6411 - val_loss: 0.6318 - val_accuracy: 0.6493
Epoch 2/35
5014/5014 - 43s - loss: 0.6285 - accuracy: 0.6508 - val_loss: 0.6192 - val_accuracy: 0.6595
Epoch 3/35
5014/5014 - 48s - loss: 0.6160 - accuracy: 0.6592 - val_loss: 0.6079 - val_accuracy: 0.6696
Epoch 4/35
5014/5014 - 52s - loss: 0.6071 - accuracy: 0.6639 - val_loss: 0.5972 - val_accuracy: 0.6707
Epoch 5/35
5014/5014 - 54s - loss: 0.6027 - accuracy: 0.6657 - val_loss: 0.5887 - val_accuracy: 0.6738
Epoch 6/35
5014/5014 - 50s - loss: 0.5972 - accuracy: 0.6667 - val_loss: 0.5909 - val_accuracy: 0.6743
Epoch 7/35
5014/5014 - 47s - loss: 0.5959 - accuracy: 0.6680 - val_loss: 0.5852 - val_accuracy: 0.6742
Epoch 8/35
5014/5014 - 42s - loss: 0.5933 - accuracy: 0.6698 - val_loss: 0.5824 - val_accuracy: 0.6794
Epoch 9/35
5014/5014 - 44s - loss: 0.5906 - accuracy: 0.6714 - val_loss: 0.5748 - val_accuracy: 0.6821
Epoch 10/35
5014/5014 - 33s - loss: 0.5885 - accuracy: 0.6723 - val_loss:

In [ ]:
loss, accuracy = model.evaluate(testX, testY)
print("Loss:" + str(loss))
print("Accuracy:" + str(accuracy))

### Conv-LSTM

In [104]:
XS.shape

(401119, 17)

In [105]:
X1 = XS.reshape((-1, 1, 17))

In [106]:
X1.shape

(401119, 1, 17)

In [107]:
(trainX, testX, trainY, testY) = train_test_split(X1, y, test_size = 0.20, random_state = 5)

In [108]:
nb_filter = 250
filter_length = 3

model = Sequential()
model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, padding='same', activation='tanh'))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.2))
model.add(LSTM(17))
model.add(Dropout(0.2))
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(trainX, trainY, epochs=35, batch_size=64, verbose=2, validation_data=(testX, testY))

Epoch 1/35
5014/5014 - 30s - loss: 0.6456 - accuracy: 0.6420 - val_loss: 0.6261 - val_accuracy: 0.6506
Epoch 2/35


KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(testX, testY)
print("Loss:" + str(loss))
print("Accuracy:" + str(accuracy))

In [118]:
nb_filter = 250
filter_length = 3

model = Sequential()
model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, padding='same', activation='tanh'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, padding='same', activation='tanh'))
model.add(MaxPooling1D(pool_size=1))
model.add(Activation('softmax'))
model.add(LSTM(17, return_sequences=True))
model.add(LSTM(17, return_sequences=True))
model.add(Dense(128, activation='tanh'))
model.add(Dense(100, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(trainX, trainY, epochs=500, batch_size=100, verbose=2, validation_data=(testX, testY))

Epoch 1/500
3209/3209 - 38s - loss: 0.6686 - accuracy: 0.6110 - val_loss: 0.6688 - val_accuracy: 0.6098
Epoch 2/500
3209/3209 - 32s - loss: 0.6479 - accuracy: 0.6361 - val_loss: 0.6250 - val_accuracy: 0.6589
Epoch 3/500
3209/3209 - 32s - loss: 0.6144 - accuracy: 0.6621 - val_loss: 0.5966 - val_accuracy: 0.6686
Epoch 4/500
3209/3209 - 33s - loss: 0.5943 - accuracy: 0.6692 - val_loss: 0.5931 - val_accuracy: 0.6701
Epoch 5/500
3209/3209 - 37s - loss: 0.5880 - accuracy: 0.6720 - val_loss: 0.5828 - val_accuracy: 0.6753
Epoch 6/500
3209/3209 - 34s - loss: 0.5843 - accuracy: 0.6746 - val_loss: 0.5791 - val_accuracy: 0.6786
Epoch 7/500
3209/3209 - 32s - loss: 0.5821 - accuracy: 0.6762 - val_loss: 0.5857 - val_accuracy: 0.6772
Epoch 8/500
3209/3209 - 32s - loss: 0.5792 - accuracy: 0.6781 - val_loss: 0.5787 - val_accuracy: 0.6780
Epoch 9/500
3209/3209 - 36s - loss: 0.5773 - accuracy: 0.6789 - val_loss: 0.5744 - val_accuracy: 0.6809
Epoch 10/500
3209/3209 - 38s - loss: 0.5756 - accuracy: 0.6799 -

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(testX, testY)
print("Loss:" + str(loss))
print("Accuracy:" + str(accuracy))

# Model Development (Multi Class)

In [119]:
y_M = Data['type']
y_M = y_M.values
y_M.shape

(401119,)

In [120]:
(trainX, testX, trainY, testY) = train_test_split(XS, y_M, test_size = 0.20, random_state = 5)

### LR

In [42]:
#Test Train split
lr = LogisticRegression(multi_class='ovr') #for multiclass
lr.fit(trainX, trainY)
y_predict = lr.predict(testX)
print(accuracy_score(testY, y_predict))
print(classification_report(testY, y_predict))

C:\Users\usman\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\usman\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

0.6086831870761866


C:\Users\usman\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    backdoor       0.00      0.00      0.00      7034
        ddos       0.37      0.05      0.08      5125
   injection       0.00      0.00      0.00      6950
      normal       0.61      0.99      0.76     48919
    password       0.00      0.00      0.00      7003
  ransomware       0.00      0.00      0.00      3184
    scanning       0.00      0.00      0.00       810
         xss       0.00      0.00      0.00      1199

    accuracy                           0.61     80224
   macro avg       0.12      0.13      0.10     80224
weighted avg       0.40      0.61      0.47     80224



In [ ]:
#Kfold Split
lr = LogisticRegression(multi_class='ovr')
cv_results = cross_val_score(lr, XS, y_M, cv=kfold, scoring='accuracy')
cv_results_mean = cv_results.mean()
cv_results_std = cv_results.std()
cv_results_name = 'LR'
print('%s: %f (%f)' % (cv_results_name, cv_results_mean, cv_results_std))

In [ ]:
#Grid Search CV
model = LogisticRegression() 

parameters = [{'solver': ['newton-cg', 'lbfgs', 'liblinear'], 'C': [100, 10, 1.0, 0.1, 0.01]}]

grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 4, n_jobs = -1)

grid_search = grid_search.fit(trainX, trainY)

print('Best Results with Grid Search:')
print(grid_search.best_score_)
print(grid_search.best_params_)

y_pred = grid_search.predict(testX)

print('\nAccuracy Score  on test data: ' + str(accuracy_score(testY, y_pred)))
print(confusion_matrix(testY,y_pred))
print(classification_report(testY,y_pred))

### LDA

In [43]:
#Test Train split
lda = LinearDiscriminantAnalysis()
lda.fit(trainX, trainY)
y_predict = lda.predict(testX)
print(accuracy_score(testY, y_predict))
print(classification_report(testY, y_predict))

0.6087081172716394


C:\Users\usman\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    backdoor       0.00      0.00      0.00      7034
        ddos       0.32      0.10      0.15      5125
   injection       0.00      0.00      0.00      6950
      normal       0.62      0.98      0.76     48919
    password       0.49      0.03      0.05      7003
  ransomware       0.00      0.00      0.00      3184
    scanning       0.14      0.14      0.14       810
         xss       0.00      0.00      0.00      1199

    accuracy                           0.61     80224
   macro avg       0.20      0.16      0.14     80224
weighted avg       0.44      0.61      0.48     80224



In [ ]:
#Kfold Split
lda = LinearDiscriminantAnalysis()
cv_results = cross_val_score(lda, XS, y_M, cv=kfold, scoring='accuracy')
cv_results_mean = cv_results.mean()
cv_results_std = cv_results.std()
cv_results_name = 'LDA'
print('%s: %f (%f)' % (cv_results_name, cv_results_mean, cv_results_std))

In [ ]:
#Grid search CV
model = LinearDiscriminantAnalysis() 

parameters = [{'solver': ['svd', 'lsqr', 'eigen']}]

grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 4, n_jobs = -1)

grid_search = grid_search.fit(trainX, trainY)

print('Best Results with Grid Search:')
print(grid_search.best_score_)
print(grid_search.best_params_)

y_pred = grid_search.predict(testX)

print('\nAccuracy Score  on test data: ' + str(accuracy_score(testY, y_pred)))
print(confusion_matrix(testY,y_pred))
print(classification_report(testY,y_pred))

### KNN

In [44]:
#Test Train split
knn = KNeighborsClassifier()
knn.fit(trainX, trainY)
y_predict = knn.predict(testX)
print(accuracy_score(testY, y_predict))
print(classification_report(testY, y_predict))

0.5679347826086957
              precision    recall  f1-score   support

    backdoor       0.17      0.55      0.26      7034
        ddos       0.61      0.35      0.45      5125
   injection       0.58      0.36      0.44      6950
      normal       0.75      0.70      0.73     48919
    password       0.67      0.26      0.37      7003
  ransomware       0.76      0.26      0.38      3184
    scanning       0.94      0.26      0.40       810
         xss       0.47      0.12      0.18      1199

    accuracy                           0.57     80224
   macro avg       0.62      0.36      0.40     80224
weighted avg       0.67      0.57      0.59     80224



In [ ]:
#Kfold Split
knn = KNeighborsClassifier()
cv_results = cross_val_score(knn, XS, y_M, cv=kfold, scoring='accuracy')
cv_results_mean = cv_results.mean()
cv_results_std = cv_results.std()
cv_results_name = 'KNN'
print('%s: %f (%f)' % (cv_results_name, cv_results_mean, cv_results_std))

In [ ]:
#Grid search CV
model = KNeighborsClassifier()

parameters = [{'n_neighbors': [range(1, 21, 2)], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']}]

grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 4, n_jobs = -1)

grid_search = grid_search.fit(trainX, trainY)

print('Best Results with Grid Search:')
print(grid_search.best_score_)
print(grid_search.best_params_)

y_pred = grid_search.predict(testX)

print('\nAccuracy Score  on test data: ' + str(accuracy_score(testY, y_pred)))
print(confusion_matrix(testY,y_pred))
print(classification_report(testY,y_pred))

### RF 

In [45]:
#Test Train split
rf = RandomForestClassifier(n_estimators=10,criterion='gini')
rf.fit(trainX, trainY)
y_predict1 = rf.predict(testX)
print(accuracy_score(testY, y_predict1))
print(classification_report(testY, y_predict1))

0.7249576186677303
              precision    recall  f1-score   support

    backdoor       0.73      0.41      0.52      7034
        ddos       0.88      0.35      0.50      5125
   injection       0.73      0.41      0.53      6950
      normal       0.72      0.95      0.82     48919
    password       0.71      0.36      0.47      7003
  ransomware       0.64      0.29      0.40      3184
    scanning       0.94      0.34      0.50       810
         xss       0.74      0.22      0.34      1199

    accuracy                           0.72     80224
   macro avg       0.76      0.42      0.51     80224
weighted avg       0.73      0.72      0.69     80224



In [ ]:
#Kfold Split
rf = RandomForestClassifier(n_estimators=10,criterion='gini')
cv_results = cross_val_score(rf, XS, y_M, cv=kfold, scoring='accuracy')
cv_results_mean = cv_results.mean()
cv_results_std = cv_results.std()
cv_results_name = 'RF'
print('%s: %f (%f)' % (cv_results_name, cv_results_mean, cv_results_std))

In [ ]:
#Grid Search CV
model = RandomForestClassifier()

parameters = [{'n_estimators': [10, 20, 50, 100], 'min_samples_split':[2, 5, 10],
              'min_samples_leaf':[1, 2, 4], 'criterion': ['gini','entropy']}]


grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 4, n_jobs = -1)

grid_search = grid_search.fit(trainX, trainY)

print('Best Results with Grid Search:')
print(grid_search.best_score_)
print(grid_search.best_params_)

y_pred = grid_search.predict(testX)

print('\nAccuracy Score  on test data: ' + str(accuracy_score(testY, y_pred)))
print(confusion_matrix(testY,y_pred))
print(classification_report(testY,y_pred))

### DT (CART) 

In [46]:
#Test Train split
dt = DecisionTreeClassifier(criterion='gini')
dt.fit(trainX, trainY)
y_predict1 = dt.predict(testX)
print(accuracy_score(testY, y_predict1))
print(classification_report(testY, y_predict1))

0.7198344635021938
              precision    recall  f1-score   support

    backdoor       0.72      0.40      0.51      7034
        ddos       0.87      0.35      0.50      5125
   injection       0.73      0.41      0.52      6950
      normal       0.72      0.95      0.82     48919
    password       0.67      0.36      0.47      7003
  ransomware       0.58      0.29      0.39      3184
    scanning       0.89      0.35      0.51       810
         xss       0.64      0.24      0.35      1199

    accuracy                           0.72     80224
   macro avg       0.73      0.42      0.51     80224
weighted avg       0.72      0.72      0.69     80224



In [ ]:
#Kfold Split
dt = DecisionTreeClassifier(criterion='gini')
cv_results = cross_val_score(dt, XS, y_M, cv=kfold, scoring='accuracy')
cv_results_mean = cv_results.mean()
cv_results_std = cv_results.std()
cv_results_name = 'DT-(CART)'
print('%s: %f (%f)' % (cv_results_name, cv_results_mean, cv_results_std))

In [ ]:
#Grid Search CV
model = DecisionTreeClassifier() 

parameters = [{'min_samples_split': [2,5,10], 'min_samples_leaf': [1,2,4], 'splitter': ['best','random'],
              'criterion': ['gini', 'entropy']}]

grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 4, n_jobs = -1)

grid_search = grid_search.fit(trainX, trainY)

print('Best Results with Grid Search:')
print(grid_search.best_score_)
print(grid_search.best_params_)

y_pred = grid_search.predict(testX)

print('\nAccuracy Score  on test data: ' + str(accuracy_score(testY, y_pred)))
print(confusion_matrix(testY,y_pred))
print(classification_report(testY,y_pred))

### NB 

In [47]:
#Test Train split
nb = GaussianNB()
nb.fit(trainX, trainY)
y_predict1 = nb.predict(testX)
print(accuracy_score(testY, y_predict1))
print(classification_report(testY, y_predict1))

0.07077682489030714


C:\Users\usman\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    backdoor       0.00      0.00      0.00      7034
        ddos       0.09      1.00      0.16      5125
   injection       0.00      0.00      0.00      6950
      normal       0.00      0.00      0.00     48919
    password       0.00      0.00      0.00      7003
  ransomware       0.04      0.14      0.06      3184
    scanning       0.01      0.14      0.02       810
         xss       0.00      0.00      0.00      1199

    accuracy                           0.07     80224
   macro avg       0.02      0.16      0.03     80224
weighted avg       0.01      0.07      0.01     80224



In [ ]:
#Kfold Split
nb = GaussianNB()
cv_results = cross_val_score(nb, XS, y_M, cv=kfold, scoring='accuracy')
cv_results_mean = cv_results.mean()
cv_results_std = cv_results.std()
cv_results_name = 'NB'
print('%s: %f (%f)' % (cv_results_name, cv_results_mean, cv_results_std))

### SVM

In [74]:
#Test Train split
svclassifier = SVC(kernel='rbf', gamma='auto')
svclassifier.fit(trainX, trainY)
y_predict = svclassifier.predict(testX)
print(accuracy_score(testY, y_predict))
print(classification_report(testY, y_predict))

0.6113257877941763


C:\Users\usman\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    backdoor       0.00      0.00      0.00      7034
        ddos       1.00      0.02      0.05      5125
   injection       0.00      0.00      0.00      6950
      normal       0.61      1.00      0.76     48919
    password       0.00      0.00      0.00      7003
  ransomware       0.00      0.00      0.00      3184
    scanning       0.00      0.00      0.00       810
         xss       0.00      0.00      0.00      1199

    accuracy                           0.61     80224
   macro avg       0.20      0.13      0.10     80224
weighted avg       0.44      0.61      0.47     80224



In [ ]:
#Kfold Split
svclassifier = SVC(kernel='rbf', gamma='auto')
cv_results = cross_val_score(svclassifier, XS, y_M, cv=kfold, scoring='accuracy')
cv_results_mean = cv_results.mean()
cv_results_std = cv_results.std()
cv_results_name = 'LDA'
print('%s: %f (%f)' % (cv_results_name, cv_results_mean, cv_results_std))

In [ ]:
#Grid Search CV
model = SVC() 

parameters = [{'C': [0.1,1,10], 'gamma':[0.01,0.5,1]}]

grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 4, n_jobs = -1)

grid_search = grid_search.fit(trainX, trainY)

print('Best Results with Grid Search:')
print(grid_search.best_score_)
print(grid_search.best_params_)

y_pred = grid_search.predict(testX)

print('\nAccuracy Score  on test data: ' + str(accuracy_score(testY, y_pred)))
print(confusion_matrix(testY,y_pred))
print(classification_report(testY,y_pred))

### LSTM

In [121]:
XS.shape

(401119, 17)

In [122]:
X1 = XS.reshape((-1,1,17))

In [123]:
X1.shape

(401119, 1, 17)

In [124]:
len(np.unique(y_M))

8

In [126]:
encoder = LabelEncoder()
encoder.fit(y_M)
y_E = encoder.transform(y_M)

In [127]:
y_E.shape

(401119,)

In [128]:
(trainX, testX, trainY, testY) = train_test_split(X1, y_E, test_size=0.20, random_state = 5)

In [139]:
# 1 LSTM Layer (input), 3 Dense Hidden Layers
model = Sequential()
model.add(LSTM(17, input_shape=(1, 17), activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(8,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(trainX, trainY, epochs=35, batch_size=64, verbose=2, validation_data=(testX, testY))

Epoch 1/35
5014/5014 - 19s - loss: 1.3339 - accuracy: 0.6108 - val_loss: 1.2940 - val_accuracy: 0.6113
Epoch 2/35


KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(testX, testY)
print("Loss:" + str(loss))
print("Accuracy:" + str(accuracy))

### CNN (Conv1D)

In [133]:
XS.shape

(401119, 17)

In [134]:
X1 = XS.reshape((-1,1,17))

In [135]:
X1.shape

(401119, 1, 17)

In [136]:
encoder = LabelEncoder()
encoder.fit(y_M)
y_E = encoder.transform(y_M)

In [137]:
(trainX, testX, trainY, testY) = train_test_split(X1, y_E, test_size = 0.20, random_state = 5)

In [138]:
nb_filter = 250
filter_length = 3

model = Sequential()
model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, padding='same', activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(8,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(trainX, trainY, epochs=35, batch_size=64, verbose=2, validation_data=(testX, testY))

Epoch 1/35
5014/5014 - 21s - loss: 1.3224 - accuracy: 0.6129 - val_loss: 1.2808 - val_accuracy: 0.6189
Epoch 2/35


KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(testX, testY)
print("Loss:" + str(loss))
print("Accuracy:" + str(accuracy))

### Conv-LSTM

In [140]:
XS.shape

(401119, 17)

In [141]:
X1 = XS.reshape((-1,1,17))

In [142]:
X1.shape

(401119, 1, 17)

In [143]:
encoder = LabelEncoder()
encoder.fit(y_M)
y_E = encoder.transform(y_M)

In [144]:
(trainX, testX, trainY, testY) = train_test_split(X1, y_E, test_size = 0.20, random_state = 5)

In [145]:
nb_filter = 250
filter_length = 3

model = Sequential()
model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, padding='same', activation='tanh'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, padding='same', activation='tanh'))
model.add(MaxPooling1D(pool_size=1))
model.add(Activation('softmax'))
model.add(LSTM(17, return_sequences=True))
model.add(LSTM(17, return_sequences=True))
model.add(Dense(128, activation='tanh'))
model.add(Dense(100, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(8,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(trainX, trainY, epochs=35, batch_size=64, verbose=2, validation_data=(testX, testY))

Epoch 1/35
5014/5014 - 49s - loss: 1.3253 - accuracy: 0.6137 - val_loss: 1.3003 - val_accuracy: 0.6167
Epoch 2/35


KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(testX, testY)
print("Loss:" + str(loss))
print("Accuracy:" + str(accuracy))